# Streamlit Load Test Analysis
This notebook analyzes the performance metrics collected from your Streamlit application to determine if it can handle 150-200 concurrent users.

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully")

In [ ]:
# Configure paths
# CHANGE THIS TO YOUR METRICS DIRECTORY
METRICS_DIR = "/path/to/your/metrics/directory"
REPORTS_DIR = "/path/to/your/reports/directory"

# Date for the CSV files (today's date)
csv_date = datetime.now().strftime("%Y%m%d")

# File paths
operations_file = os.path.join(METRICS_DIR, f"operation_metrics_{csv_date}.csv")
checkpoints_file = os.path.join(METRICS_DIR, f"checkpoint_metrics_{csv_date}.csv")
summary_file = os.path.join(METRICS_DIR, f"summary_metrics_{csv_date}.csv")

print(f"Metrics directory: {METRICS_DIR}")
print(f"Looking for files from date: {csv_date}")
print(f"\nChecking for files:")
for file in [operations_file, checkpoints_file, summary_file]:
    exists = os.path.exists(file)
    print(f"  {os.path.basename(file)}: {'Found' if exists else 'Not found'}")

## 2. Load Data

In [ ]:
# Load CSV files
df_operations = None
df_checkpoints = None
df_summary = None

if os.path.exists(operations_file):
    df_operations = pd.read_csv(operations_file)
    df_operations['timestamp'] = pd.to_datetime(df_operations['timestamp'])
    print(f"Operations data loaded: {len(df_operations)} records")
else:
    print("Warning: Operations file not found")

if os.path.exists(checkpoints_file):
    df_checkpoints = pd.read_csv(checkpoints_file)
    df_checkpoints['timestamp'] = pd.to_datetime(df_checkpoints['timestamp'])
    print(f"Checkpoints data loaded: {len(df_checkpoints)} records")
else:
    print("Warning: Checkpoints file not found")

if os.path.exists(summary_file):
    df_summary = pd.read_csv(summary_file)
    df_summary['timestamp'] = pd.to_datetime(df_summary['timestamp'])
    print(f"Summary data loaded: {len(df_summary)} records")
else:
    print("Warning: Summary file not found")

## 3. Generate Load Test Report

In [ ]:
# Import the report generator
from memory_compute_tracker import generate_load_test_report

# Generate comprehensive report
report_path = generate_load_test_report(
    metrics_dir=METRICS_DIR,
    output_dir=REPORTS_DIR
)

print(f"Report generated: {report_path}")

# Display report content
with open(report_path, 'r') as f:
    print("\n" + "="*60)
    print(f.read())
    print("="*60)

## 4. Operation Performance Analysis

In [ ]:
if df_operations is not None and not df_operations.empty:
    # Summary statistics by operation
    operation_summary = df_operations.groupby('operation').agg({
        'duration_seconds': ['count', 'mean', 'std', 'min', 'max'],
        'peak_mb': ['mean', 'max'],
        'final_increase_mb': ['mean', 'max'],
        'avg_cpu_percent': ['mean', 'max'],
        'had_spike': 'sum'
    }).round(2)
    
    operation_summary.columns = ['_'.join(col).strip() for col in operation_summary.columns]
    operation_summary = operation_summary.rename(columns={'duration_seconds_count': 'call_count'})
    
    print("OPERATION PERFORMANCE SUMMARY")
    print("="*60)
    display(operation_summary)
    
    # Identify problematic operations
    print("\nPROBLEMATIC OPERATIONS:")
    print("-"*40)
    
    # Slow operations (>5 seconds average)
    slow_ops = operation_summary[operation_summary['duration_seconds_mean'] > 5]
    if not slow_ops.empty:
        print("\nSlow operations (>5s average):")
        for op in slow_ops.index:
            print(f"  - {op}: {slow_ops.loc[op, 'duration_seconds_mean']:.2f}s average")
    
    # Memory intensive operations (>100MB peak average)
    memory_ops = operation_summary[operation_summary['peak_mb_mean'] > 100]
    if not memory_ops.empty:
        print("\nMemory intensive operations (>100MB average peak):")
        for op in memory_ops.index:
            print(f"  - {op}: {memory_ops.loc[op, 'peak_mb_mean']:.0f}MB average peak")
    
    # Operations with memory spikes
    spike_ops = operation_summary[operation_summary['had_spike_sum'] > 0]
    if not spike_ops.empty:
        print("\nOperations with memory spikes:")
        for op in spike_ops.index:
            print(f"  - {op}: {spike_ops.loc[op, 'had_spike_sum']:.0f} spikes")
else:
    print("No operations data available")

## 5. Memory Usage Visualization

In [ ]:
if df_summary is not None and not df_summary.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: Memory over time
    ax1 = axes[0, 0]
    ax1.plot(df_summary['timestamp'], df_summary['current_memory_mb'], label='Current', linewidth=2)
    ax1.plot(df_summary['timestamp'], df_summary['peak_memory_mb'], label='Peak', linewidth=2, alpha=0.7)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Memory (MB)')
    ax1.set_title('Memory Usage Over Time')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Active sessions over time
    ax2 = axes[0, 1]
    ax2.plot(df_summary['timestamp'], df_summary['active_sessions'], color='green', linewidth=2)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Active Sessions')
    ax2.set_title('Concurrent Sessions Over Time')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: CPU usage over time
    ax3 = axes[1, 0]
    ax3.plot(df_summary['timestamp'], df_summary['cpu_percent'], color='red', linewidth=2)
    ax3.axhline(y=80, color='orange', linestyle='--', label='Warning threshold')
    ax3.set_xlabel('Time')
    ax3.set_ylabel('CPU %')
    ax3.set_title('CPU Usage Over Time')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: 150-user projection over time
    ax4 = axes[1, 1]
    ax4.plot(df_summary['timestamp'], df_summary['projection_150_users_gb'], color='purple', linewidth=2)
    ax4.axhline(y=200, color='red', linestyle='--', label='Fail threshold (200GB)')
    ax4.axhline(y=150, color='orange', linestyle='--', label='Warning threshold (150GB)')
    ax4.set_xlabel('Time')
    ax4.set_ylabel('Projected Memory (GB)')
    ax4.set_title('150-User Memory Projection')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\nMEMORY STATISTICS:")
    print("="*40)
    print(f"Current memory range: {df_summary['current_memory_mb'].min():.0f} - {df_summary['current_memory_mb'].max():.0f} MB")
    print(f"Peak memory reached: {df_summary['peak_memory_mb'].max():.0f} MB")
    print(f"Average memory: {df_summary['current_memory_mb'].mean():.0f} MB")
    print(f"\nMax concurrent sessions: {df_summary['active_sessions'].max()}")
    print(f"Average CPU usage: {df_summary['cpu_percent'].mean():.1f}%")
    print(f"Peak CPU usage: {df_summary['cpu_percent'].max():.1f}%")
else:
    print("No summary data available for visualization")

## 6. Operation Performance Distribution

In [ ]:
if df_operations is not None and not df_operations.empty:
    # Get unique operations
    operations = df_operations['operation'].unique()
    
    if len(operations) > 0:
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Duration distribution
        ax1 = axes[0, 0]
        for op in operations:
            op_data = df_operations[df_operations['operation'] == op]['duration_seconds']
            ax1.hist(op_data, alpha=0.5, label=op, bins=20)
        ax1.set_xlabel('Duration (seconds)')
        ax1.set_ylabel('Frequency')
        ax1.set_title('Operation Duration Distribution')
        ax1.legend()
        
        # Memory increase distribution
        ax2 = axes[0, 1]
        for op in operations:
            op_data = df_operations[df_operations['operation'] == op]['final_increase_mb']
            ax2.hist(op_data, alpha=0.5, label=op, bins=20)
        ax2.set_xlabel('Memory Increase (MB)')
        ax2.set_ylabel('Frequency')
        ax2.set_title('Memory Increase Distribution')
        ax2.legend()
        
        # Peak memory by operation
        ax3 = axes[1, 0]
        peak_by_op = df_operations.groupby('operation')['peak_mb'].agg(['mean', 'max'])
        x_pos = np.arange(len(peak_by_op.index))
        ax3.bar(x_pos - 0.2, peak_by_op['mean'], 0.4, label='Average Peak', alpha=0.8)
        ax3.bar(x_pos + 0.2, peak_by_op['max'], 0.4, label='Maximum Peak', alpha=0.8)
        ax3.set_xlabel('Operation')
        ax3.set_ylabel('Memory (MB)')
        ax3.set_title('Peak Memory by Operation')
        ax3.set_xticks(x_pos)
        ax3.set_xticklabels(peak_by_op.index, rotation=45, ha='right')
        ax3.legend()
        
        # CPU usage by operation
        ax4 = axes[1, 1]
        cpu_by_op = df_operations.groupby('operation')['avg_cpu_percent'].agg(['mean', 'max'])
        x_pos = np.arange(len(cpu_by_op.index))
        ax4.bar(x_pos - 0.2, cpu_by_op['mean'], 0.4, label='Average CPU', alpha=0.8)
        ax4.bar(x_pos + 0.2, cpu_by_op['max'], 0.4, label='Maximum CPU', alpha=0.8)
        ax4.set_xlabel('Operation')
        ax4.set_ylabel('CPU %')
        ax4.set_title('CPU Usage by Operation')
        ax4.set_xticks(x_pos)
        ax4.set_xticklabels(cpu_by_op.index, rotation=45, ha='right')
        ax4.legend()
        
        plt.tight_layout()
        plt.show()
else:
    print("No operations data available for distribution analysis")

## 7. Load Capacity Assessment

In [ ]:
if df_summary is not None and not df_summary.empty:
    print("LOAD CAPACITY ASSESSMENT FOR 150 USERS")
    print("="*60)
    
    # Get latest metrics
    latest = df_summary.iloc[-1]
    
    # Calculate statistics
    avg_projection = df_summary['projection_150_users_gb'].mean()
    max_projection = df_summary['projection_150_users_gb'].max()
    min_projection = df_summary['projection_150_users_gb'].min()
    std_projection = df_summary['projection_150_users_gb'].std()
    
    print(f"\nProjection Statistics:")
    print(f"  Latest projection: {latest['projection_150_users_gb']:.2f} GB")
    print(f"  Average projection: {avg_projection:.2f} GB")
    print(f"  Minimum projection: {min_projection:.2f} GB")
    print(f"  Maximum projection: {max_projection:.2f} GB")
    print(f"  Standard deviation: {std_projection:.2f} GB")
    
    print(f"\nPer-User Estimates:")
    print(f"  Current estimate: {latest['estimated_per_user_mb']:.1f} MB per user")
    print(f"  Max observed: {df_summary['estimated_per_user_mb'].max():.1f} MB per user")
    
    print(f"\nTest Conditions:")
    print(f"  Max concurrent sessions tested: {df_summary['active_sessions'].max()}")
    print(f"  Total operations: {latest['total_operations']}")
    
    # Make assessment
    print("\n" + "="*60)
    print("FINAL ASSESSMENT:")
    print("="*60)
    
    # Use worst-case (maximum) projection for conservative assessment
    if max_projection > 200:
        print("❌ FAIL - Application will NOT handle 150 users")
        print(f"\nReason: Worst-case memory projection ({max_projection:.1f} GB) exceeds 200 GB limit")
        print("\nRequired Actions:")
        print("  1. Implement @st.cache_resource on JSON loading (critical)")
        print("  2. Reduce session state memory usage")
        print("  3. Optimize memory-intensive operations")
        
    elif max_projection > 150:
        print("⚠️  WARNING - Application may struggle with 150 users")
        print(f"\nReason: Worst-case memory projection ({max_projection:.1f} GB) is in warning zone")
        print("\nRecommended Actions:")
        print("  1. Monitor closely during deployment")
        print("  2. Consider implementing caching optimizations")
        print("  3. Have scaling plan ready")
        
    else:
        print("✅ PASS - Application should handle 150 users")
        print(f"\nWorst-case projection: {max_projection:.1f} GB (well within limits)")
        print("\nRecommendations:")
        print("  1. Proceed with deployment")
        print("  2. Monitor initial production usage")
        print("  3. Set up alerts for memory > 100GB")
    
    # Additional warnings
    if df_summary['cpu_percent'].max() > 80:
        print("\n⚠️  Additional Warning: High CPU usage detected")
        print(f"  Peak CPU: {df_summary['cpu_percent'].max():.1f}%")
        print("  CPU may become a bottleneck before memory")
        
else:
    print("No summary data available for assessment")

## 8. Export Results

In [ ]:
# Create a summary dataframe for export
if df_summary is not None and not df_summary.empty:
    export_data = {
        'Metric': [
            'Latest Memory Projection (GB)',
            'Average Memory Projection (GB)',
            'Maximum Memory Projection (GB)',
            'Estimated Per User (MB)',
            'Peak Memory Observed (MB)',
            'Max Concurrent Sessions',
            'Total Operations',
            'Average CPU %',
            'Peak CPU %',
            'Assessment'
        ],
        'Value': [
            f"{df_summary['projection_150_users_gb'].iloc[-1]:.2f}",
            f"{df_summary['projection_150_users_gb'].mean():.2f}",
            f"{df_summary['projection_150_users_gb'].max():.2f}",
            f"{df_summary['estimated_per_user_mb'].iloc[-1]:.1f}",
            f"{df_summary['peak_memory_mb'].max():.0f}",
            f"{df_summary['active_sessions'].max()}",
            f"{df_summary['total_operations'].max()}",
            f"{df_summary['cpu_percent'].mean():.1f}",
            f"{df_summary['cpu_percent'].max():.1f}",
            "PASS" if df_summary['projection_150_users_gb'].max() < 150 else 
            "WARNING" if df_summary['projection_150_users_gb'].max() < 200 else "FAIL"
        ]
    }
    
    summary_df = pd.DataFrame(export_data)
    
    # Save to CSV
    export_file = os.path.join(REPORTS_DIR, f"load_test_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
    summary_df.to_csv(export_file, index=False)
    
    print("Summary Results:")
    print("="*40)
    display(summary_df)
    print(f"\nResults exported to: {export_file}")
else:
    print("No data available for export")

## 9. Recommendations

In [ ]:
print("OPTIMIZATION RECOMMENDATIONS")
print("="*60)

if df_operations is not None and not df_operations.empty:
    # Find the most resource-intensive operations
    top_memory_ops = df_operations.groupby('operation')['peak_mb'].max().nlargest(3)
    top_time_ops = df_operations.groupby('operation')['duration_seconds'].mean().nlargest(3)
    
    print("\n1. OPTIMIZE THESE OPERATIONS FIRST:")
    print("-"*40)
    
    print("\nHighest Memory Usage:")
    for op, mem in top_memory_ops.items():
        print(f"  - {op}: {mem:.0f} MB peak")
        if 'json' in op.lower():
            print("    → Add @st.cache_resource decorator")
        elif 'pdf' in op.lower():
            print("    → Process in chunks, clear buffers after use")
        elif 'llm' in op.lower() or 'openai' in op.lower():
            print("    → Reduce context size, implement streaming")
    
    print("\nSlowest Operations:")
    for op, time in top_time_ops.items():
        print(f"  - {op}: {time:.2f} seconds average")
        if time > 5:
            print("    → Add retry logic and timeout handling")

if df_summary is not None and not df_summary.empty:
    max_projection = df_summary['projection_150_users_gb'].max()
    
    print("\n2. CRITICAL FIXES BASED ON PROJECTION:")
    print("-"*40)
    
    if max_projection > 150:
        print("\nIMPLEMENT IMMEDIATELY:")
        print("  1. JSON Caching with @st.cache_resource (saves ~99% memory)")
        print("  2. Add retry logic to Azure OpenAI calls")
        print("  3. Implement concurrency limits (semaphores)")
        print("  4. Clear unnecessary session state data")
    else:
        print("\nOPTIONAL OPTIMIZATIONS:")
        print("  1. Implement request queuing for better load distribution")
        print("  2. Add monitoring alerts for memory > 100GB")
        print("  3. Consider horizontal scaling if load exceeds 200 users")

print("\n3. TESTING RECOMMENDATIONS:")
print("-"*40)
print("  - Test with at least 20-30 concurrent users")
print("  - Monitor for at least 30 minutes")
print("  - Test all major operations multiple times")
print("  - Verify caching is working (second loads should be instant)")

print("\n" + "="*60)
print("Analysis complete. Review the report and implement recommended fixes.")